- https://tianchi.aliyun.com/competition/entrance/531842/information
- https://datawhalechina.github.io/fun-rec/#/ch03/ch3.1/markdown/ch3.1.1
- https://www.jianshu.com/p/f306a37a7374https://www.jianshu.com/p/f306a37a7374

In [1]:
# import packages
import time, math, os
from tqdm import tqdm
import gc
import pickle
import random
from datetime import datetime
from operator import itemgetter
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
warnings.filterwarnings('ignore')

In [6]:
DATA_PATH = './data/'
SAVE_PATH = './tmp_results/'

# df节省内存函数

In [3]:
# 节约内存的一个标配函数
def reduce_mem(df):
    starttime = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction),time spend:{:2.2f} min'.format(end_mem,
                                                                                                           100*(start_mem-end_mem)/start_mem,
                                                                                                           (time.time()-starttime)/60))
    return df


# Exploration

In [21]:
a = pd.read_csv(os.path.join(DATA_PATH, 'articles.csv'), nrows=10)
a

,article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250
3,3,1,1408468313000,230
4,4,1,1407071171000,162
5,5,1,1407413929000,196
6,6,1,1409896802000,203
7,7,1,1412559620000,154
8,8,1,1414351550000,209
9,9,1,1412526792000,181


In [22]:
a_emb = pd.read_csv(os.path.join(DATA_PATH, 'articles_emb.csv'), nrows=10)
a_emb

,article_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_240,emb_241,emb_242,emb_243,emb_244,emb_245,emb_246,emb_247,emb_248,emb_249
0,0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,...,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,...,-0.487843,0.823124,0.412688,-0.338654,0.320786,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,...,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284
3,3,-0.740843,-0.975749,0.391698,0.641738,-0.268645,0.191745,-0.825593,-0.710591,-0.040099,...,0.271535,0.036040,0.480029,-0.763173,0.022627,0.565165,-0.910286,-0.537838,0.243541,-0.885329
4,4,-0.279052,-0.972315,0.685374,0.113056,0.238315,0.271913,-0.568816,0.341194,-0.600554,...,0.238286,0.809268,0.427521,-0.615932,-0.503697,0.614450,-0.917760,-0.424061,0.185484,-0.580292
5,5,-0.724885,-0.974610,0.114743,0.135984,-0.059124,0.139286,-0.720786,-0.311302,-0.543600,...,0.430414,0.492010,0.650828,-0.627942,0.562439,0.792561,-0.877570,-0.002212,0.426586,-0.065801
6,6,-0.161707,-0.967413,0.637266,0.249817,0.471958,0.391854,-0.815684,-0.296389,-0.804284,...,0.020420,0.509749,0.559321,-0.794207,-0.366995,0.616669,-0.626267,-0.452933,0.166825,-0.728330
7,7,-0.104534,-0.965899,0.800449,0.351384,-0.389743,0.421198,-0.600862,-0.021568,-0.265164,...,0.124667,0.545793,0.372644,-0.510156,-0.217100,0.676449,-0.852480,0.198584,0.243181,-0.596663
8,8,-0.417274,-0.977311,0.054967,0.351192,0.049049,-0.460898,-0.443555,0.154505,0.018687,...,0.601953,0.756593,-0.257438,-0.130827,-0.061200,0.438138,-0.719562,-0.744506,0.163397,-0.499539
9,9,-0.348442,-0.966953,0.682574,-0.400164,0.646688,0.085186,-0.636333,-0.351318,-0.542529,...,0.220249,0.025085,0.510007,-0.832078,-0.555277,0.317832,-0.576926,-0.671720,0.304353,-0.721567


In [23]:
c = pd.read_csv(os.path.join(DATA_PATH, 'train_click_log.csv'), nrows=10)
c

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5
5,199998,144978,1507029804858,4,1,17,1,25,5
6,199998,285371,1507029834858,4,1,17,1,25,5
7,199997,224171,1507029683061,4,1,17,1,16,1
8,199997,223931,1507029713061,4,1,17,1,16,1
9,199996,299697,1507029600494,4,1,17,1,25,2


# 读取采样或全量数据

In [7]:
# debug模式：从训练集中划出一部分数据来调试代码
def get_all_click_sample(data_path, sample_nums=10000):
    """
        训练集中采样一部分数据调试
        data_path: 原数据的存储路径
        sample_nums: 采样数目（这里由于机器的内存限制，可以采样用户做）
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click

# 读取点击数据，这里分成线上和线下，如果是为了获取线上提交结果应该讲测试集中的点击数据合并到总的数据中
# 如果是为了线下验证模型的有效性或者特征的有效性，可以只使用训练集
def get_all_click_df(data_path=DATA_PATH, offline=True):
    if offline:
        all_click = pd.read_csv(data_path + 'train_click_log.csv')
    else:
        trn_click = pd.read_csv(data_path + 'train_click_log.csv')
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')

        all_click = trn_click.append(tst_click)
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click


In [8]:
# 全量训练集
all_click_df = get_all_click_df(offline=False)

In [24]:
all_click_df.shape

(1630633, 9)

In [25]:
all_click_df.head()

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [26]:
train = pd.read_csv(os.path.join(DATA_PATH, 'train_click_log.csv'))
train.shape

(1112623, 9)

In [222]:
train = pd.read_csv(os.path.join(DATA_PATH, 'train_click_log.csv'))
test = pd.read_csv(os.path.join(DATA_PATH, 'testA_click_log.csv'))
total = train.append(test)

In [223]:
total.shape

(1630633, 9)

In [224]:
train.user_id.nunique()

200000

In [28]:
train.click_article_id.nunique()

31116

In [31]:
train[train.duplicated(['user_id', 'click_article_id', 'click_timestamp'], keep=False)].sort_values(['user_id', 'click_article_id'])

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type


In [32]:
all_click_df[all_click_df.duplicated(['user_id', 'click_article_id', 'click_timestamp'], keep=False)]

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type


# 获取 用户 - 文章 - 点击时间字典

In [125]:
# 根据点击时间获取用户的点击文章序列   {user1: {item1: time1, item2: time2..}...}
# @pysnooper.snoop()
def make_item_time_pair(df):
    return list(zip(df['click_article_id'], df['click_timestamp']))

# @pysnooper.snoop()
def get_user_item_time(click_df):
    
    click_df = click_df.sort_values('click_timestamp')
    
    
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(lambda x: make_item_time_pair(x))\
                                                            .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict


In [33]:
u_i_t_dict = get_user_item_time(train)

In [34]:
len(u_i_t_dict)

200000

In [35]:
train.user_id.sample(1)

855947    79990
Name: user_id, dtype: int64

In [37]:
# u_i_t_dict[79990]

In [225]:
train.columns = ['u', 'a', 'ts', 'env', 'device', 'os', 'country', 'region', 'referrer']
test.columns = ['u', 'a', 'ts', 'env', 'device', 'os', 'country', 'region', 'referrer']
total.columns = ['u', 'a', 'ts', 'env', 'device', 'os', 'country', 'region', 'referrer']

# 获取点击最多的Topk个文章

In [10]:
# 获取近期点击最多的文章
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click


In [55]:
train.a.value_counts().index[:5]

Int64Index([234698, 123909, 336221, 96210, 183176], dtype='int64')

In [56]:
train[train.a == 234698].shape

(11886, 9)

# itemCF的物品相似度计算

In [67]:
import pysnooper

In [76]:
# !mkdir log

In [118]:
# pysnooper.snoop('log/itemcf_sim.log')
def itemcf_sim(df):
    """
        文章与文章之间的相似性矩阵计算
        :param df: 数据表
        :item_created_time_dict:  文章创建时间的字典
        return : 文章与文章的相似性矩阵
        思路: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    # {u1: [(a1, ts1), (a2, ts2)]}
    user_item_time_dict = get_user_item_time(df)
    
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)
    for user, item_time_list in tqdm(user_item_time_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素

        for i, i_click_time in item_time_list:
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0)
                
                i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)
                
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim.pkl', 'wb'))
    
    return i2i_sim_


In [83]:
i2i_sim = itemcf_sim(all_click_df)

100%|██████████████████████████████████████████████████████████| 250000/250000 [00:16<00:00, 15308.63it/s]


In [58]:
type(i2i_sim)

dict

In [64]:
all_click_df.click_article_id.nunique()

35380

In [63]:
type(i2i_sim), len(i2i_sim), len(i2i_sim.values())

(dict, 35380, 35380)

In [61]:
len(i2i_sim[30760])

1775

In [66]:
i2i_sim[30760][157507]

0.008340645572844295

In [90]:
# import logging

# logger = logging.getLogger()

# logger.setLevel(logging.ERROR)

In [240]:
def itemcf_sim1(df):
    # {u1: [(a1, ts1), (a2, ts2)]}
    user_item_time_dict = get_user_item_time(df)
    
    # with pysnooper.snoop(depth=4):
    # 计算物品相似度
    i2i_sim = {}
    item_cnt = defaultdict(int)

    for user, item_time_list in tqdm(user_item_time_dict.items()):
        # 在基于商品的协同过滤优化的时候可以考虑时间因素

        for i, i_click_time in item_time_list:
            item_cnt[i] += 1
            i2i_sim.setdefault(i, {})
            for j, j_click_time in item_time_list:
                if(i == j):
                    continue
                i2i_sim[i].setdefault(j, 0)

                # 你可以看一下项亮老师的推荐系统实践，里面有详细的说明，这里主要是引入逆用户频率(IUF, Inverse User Frequence)，
                # 认为活跃用户对物品相似度的贡献应该小于不活跃用户，所以这个IUF是用来修正物品相似度的

                # https://www.jianshu.com/p/f306a37a7374
                i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)

    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
    
    # 将得到的相似性矩阵保存到本地
    pickle.dump(i2i_sim_, open(save_path + 'itemcf_i2i_sim1.pkl', 'wb'))
    
    return i2i_sim_


In [169]:
d = all_click_df.head(5)
d

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [170]:
itemcf_sim1(d)

{157770: {96613: 0.9102392266268373},
 96613: {157770: 0.9102392266268373},
 160417: {5408: 0.7213475204444817, 50823: 0.7213475204444817},
 5408: {160417: 0.7213475204444817, 50823: 0.7213475204444817},
 50823: {160417: 0.7213475204444817, 5408: 0.7213475204444817}}

In [177]:
d = train.head()
d

,u,a,ts,env,device,os,country,region,referrer
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5


In [265]:
def self_itemcf_sim(d):    
    u2items = {}
    d = d.sort_values("ts")
    u2items = d.groupby('u')['a'].apply(lambda x: x.values.tolist()).to_dict()
    # print('u2items:')
    # print(u2items)
    # print('-'*30)
    
    i2i_sim = defaultdict(dict)
    i_cnt = defaultdict(int)
    for u, items in tqdm(u2items.items()):
        for i in items:
            i2i_sim.setdefault(i, {})
            i_cnt[i] += 1
            for j in items:
                if i == j:
                    continue
                
                i2i_sim[i].setdefault(j, 0)
                i2i_sim[i][j] += 1 / np.log(len(items) + 1) # IUF
    
    i2i_sim_ = i2i_sim.copy()
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_[i][j] = wij / np.sqrt(i_cnt[i] * i_cnt[j])
    
    
    # print('i2i_sim:')
    # print(i2i_sim_)
    # print('-'*30)
    # print('i_cnt:')
    # print(i_cnt)
    
    return i2i_sim_, i_cnt

In [210]:
offical_i2i_sim = itemcf_sim(all_click_df)

100%|██████████████████████████████████████████████████████████| 250000/250000 [00:15<00:00, 15673.57it/s]


In [266]:
self_i2i_sim, i_cnt = self_itemcf_sim(total)

100%|███████████████████████████████████████████████████████████| 250000/250000 [00:40<00:00, 6121.54it/s]


In [267]:
assert len(offical_i2i_sim) == len(self_i2i_sim)

In [268]:
assert offical_i2i_sim == self_i2i_sim

In [262]:
offical_i2i_sim[30760][157507]

0.008340645572844295

In [263]:
offical_i2i_sim[157507][30760]

0.008340645572844295

```text
  a b c
a   1 1
b 1   1
c 1 1
```

In [277]:
sorted([(k, v) for k, v in i_cnt.items()], key=lambda x: -x[1])[:5]

[(272143, 15935),
 (234698, 15666),
 (123909, 15383),
 (336221, 15170),
 (96210, 14009)]

In [283]:
total[total.a == 272143].shape[0] == 15935

True

In [290]:
2 / np.sqrt(12)

0.5773502691896258

# itemCF 的文章推荐

In [16]:
# 基于商品的召回i2i
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click):
    """
        基于文章协同过滤的召回
        :param user_id: 用户id
        :param user_item_time_dict: 字典, 根据点击时间获取用户的点击文章序列   {user1: {item1: time1, item2: time2..}...}
        :param i2i_sim: 字典，文章相似性矩阵
        :param sim_item_topk: 整数， 选择与当前文章最相似的前k篇文章
        :param recall_item_num: 整数， 最后的召回文章数量
        :param item_topk_click: 列表，点击次数最多的文章列表，用户召回补全        
        return: 召回的文章列表 {item1:score1, item2: score2...}
        注意: 基于物品的协同过滤(详细请参考上一期推荐系统基础的组队学习)， 在多路召回部分会加上关联规则的召回策略
    """
    
    # 获取用户历史交互的文章
    user_hist_items = user_item_time_dict[user_id]
    
    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items:
                continue
                
            item_rank.setdefault(j, 0)
            item_rank[j] +=  wij
    
    # 不足10个，用热门商品补全
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items(): # 填充的item应该不在原来的列表中
                continue
            item_rank[item] = - i - 100 # 随便给个负数就行
            if len(item_rank) == recall_item_num:
                break
    
    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]
        
    return item_rank


# 给每个用户根据物品的协同过滤推荐文章

In [17]:
# 定义
user_recall_items_dict = defaultdict(dict)

# 获取 用户 - 文章 - 点击时间的字典
user_item_time_dict = get_user_item_time(all_click_df)

# 去取文章相似度
i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))

# 相似文章的数量
sim_item_topk = 10

# 召回文章数量
recall_item_num = 10

# 用户热度补全
item_topk_click = get_item_topk_click(all_click_df, k=50)

for user in tqdm(all_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, 
                                                        sim_item_topk, recall_item_num, item_topk_click)


100%|████████████████████████████████████████████████████████████| 250000/250000 [24:33<00:00, 169.71it/s]


In [292]:
len(user_recall_items_dict)

250000

In [295]:
all_click_df.groupby('user_id')['click_article_id'].nunique().sort_values()

user_id
216228      1
228152      1
228159      1
228160      1
228162      1
         ... 
249586    241
240519    251
239302    265
242546    759
249322    923
Name: click_article_id, Length: 250000, dtype: int64

In [303]:
for u, i_and_score in user_recall_items_dict.items():
    if any([e[1] < 0 for e in i_and_score]):
        print(u)
        print(i_and_score)
        break

198986
[(282732, 0.9102392266268373), (286386, 0.9102392266268373), (272143, -100), (234698, -101), (123909, -102), (336221, -103), (96210, -104), (336223, -105), (183176, -106), (168623, -107)]


In [304]:
offical_i2i_sim[282732]

{286386: 0.9102392266268373}

In [306]:
total[total.a.isin([282732, 286386])]

,u,a,ts,env,device,os,country,region,referrer
2837,198986,286386,1507031156410,4,1,17,1,25,5
2838,198986,282732,1507031186410,4,1,17,1,25,5


In [314]:
(1 / np.log(1 + 2)) / np.sqrt(1) == offical_i2i_sim[282732][286386]

True

# 召回字典转换成df

In [18]:
# 将字典的形式转换成df
user_item_score_list = []

for user, items in tqdm(user_recall_items_dict.items()):
    for item, score in items:
        user_item_score_list.append([user, item, score])

recall_df = pd.DataFrame(user_item_score_list, columns=['user_id', 'click_article_id', 'pred_score'])


100%|██████████████████████████████████████████████████████████| 250000/250000 [00:03<00:00, 81784.31it/s]


In [315]:
recall_df.head()

,user_id,click_article_id,pred_score
0,199999,276970,0.172377
1,199999,158536,0.106969
2,199999,168623,0.103572
3,199999,123909,0.103572
4,199999,286321,0.097774


In [316]:
recall_df.shape

(2500000, 3)

In [322]:
clicked_articles = total[total.u == 199999].a.values.tolist()
clicked_articles

[160417,
 5408,
 50823,
 123909,
 168623,
 161191,
 42223,
 70214,
 156843,
 159019,
 218355]

In [321]:
recall_df[recall_df.user_id == 199999].click_article_id.values.tolist()

[276970, 158536, 168623, 123909, 286321, 108855, 162655, 50864, 123818, 156625]

# 生成提交文件

In [19]:
# 生成提交文件
def submit(recall_df, topk=5, model_name=None):
    recall_df = recall_df.sort_values(by=['user_id', 'pred_score'])
    recall_df['rank'] = recall_df.groupby(['user_id'])['pred_score'].rank(ascending=False, method='first')
    
    # 判断是不是每个用户都有5篇文章及以上
    tmp = recall_df.groupby('user_id').apply(lambda x: x['rank'].max())
    assert tmp.min() >= topk
    
    del recall_df['pred_score']
    submit = recall_df[recall_df['rank'] <= topk].set_index(['user_id', 'rank']).unstack(-1).reset_index()
    
    submit.columns = [int(col) if isinstance(col, int) else col for col in submit.columns.droplevel(0)]
    # 按照提交格式定义列名
    submit = submit.rename(columns={'': 'user_id', 1: 'article_1', 2: 'article_2', 
                                                  3: 'article_3', 4: 'article_4', 5: 'article_5'})
    
    save_name = save_path + model_name + '_' + datetime.today().strftime('%m-%d') + '.csv'
    submit.to_csv(save_name, index=False, header=True)


In [20]:
# 获取测试集
tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
tst_users = tst_click['user_id'].unique()

# 从所有的召回数据中将测试集中的用户选出来
tst_recall = recall_df[recall_df['user_id'].isin(tst_users)]

# 生成提交文件
submit(tst_recall, topk=5, model_name='itemcf_baseline')


In [323]:
pd.merge(train, test, how='inner', on='u')

,u,a_x,ts_x,env_x,device_x,os_x,country_x,region_x,referrer_x,a_y,ts_y,env_y,device_y,os_y,country_y,region_y,referrer_y
